# **Notebook Title: National Data Buoy Center Timeseries Plots**
## This notebook performs the following task(s):
> - #### Reads historical buoy data from the National Data Buoy Center (NDBC) into a pandas DataFrame based on some user defined specifications
> - #### Converts the pandas DataFrame into an xarray Dataset
> - #### Indexes the xarray dataset based on date/time, and then grabs the variables we want within the data/time index specified
> - #### Plots the buoy timeseries values following a specific format

## Import packages
#### **Links to documentation for packages:**
> - #### [numpy](https://numpy.org/doc/1.21/) | [xarray](https://docs.xarray.dev/en/stable/) | [pandas](https://pandas.pydata.org/pandas-docs/version/1.3.5/) | [matplotlib](https://matplotlib.org/3.5.3/index.html) | 
> - #### Note #1: Package documentation versions linked above may not correspond to the exact package version used for this analysis.
> - #### Note #2: Comments are also included in the actual code cells. Commented links above certain pieces of code are provided to help show readers/users of this code where I got some of my ideas/code from. This attempt to document my development of the code is not exhaustive, and may include links that are for extremely basic lines of code. It is possible that there may still be snippets of code here that I simply grabbed off the internet, from places like StackOverflow, without attributing any users or including a link.

In [1]:
#-------------------------------------------------
#Import packages
import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
#-------------------------------------------------

## Index variables by time and grab variables of interest for plotting

#### **Notes**

> - #### Please read in-line comments for additional details about this code cell.

In [ ]:
#-------------------------------------------------
#Define start and end times for indexing
#Recall that the NDBC data used is generally given in the UTC timezone
start_time = '2023-10-11 14:00'
end_time   = '2023-10-12 14:00'

#Index entire xarray Dataset by start and end times
xr_dataset_date_indexed = xr_dataset.sel(date=slice(start_time, end_time))
#-------------------------------------------------
#Grab data variable values that we want for plotting
wind_dir_deg  = xr_dataset_date_indexed['wind_dir_deg']
wind_speed_ms = xr_dataset_date_indexed['wind_spd_ms']
wind_gust_ms  = xr_dataset_date_indexed['wind_gst_ms']
air_temp_c    = xr_dataset_date_indexed['air_temp_c']
water_temp_c  = xr_dataset_date_indexed['water_temp_c']
dewpoint_c    = xr_dataset_date_indexed['dewpoint_c']
#-------------------------------------------------

In [26]:
ds = xr.open_dataset('ndbc_historical_1994_to_2023_46053')

wave_height = ds['wave_height_m']
wave_height_djf = wave_height.sel(date=wave_height.date.dt.season=='DJF')
wave_height_mean_djf = wave_height_djf.groupby(wave_height_djf.date.dt.year).mean('date')

x = np.arange(wave_height_mean_djf.size)
y = wave_height_mean_djf.values

fit = np.polyfit(x, y, deg=1)
fit_function = np.poly1d(fit)

fig, ax = plt.subplots()
ax.plot(wave_height_mean_djf.year, wave_height_mean_djf)
ax.plot(wave_height_mean_djf.year, fit_function(x))

## Plot variables of interest on a subplot type figure

#### **Notes**

> - #### None

In [ ]:
#-------------------------------------------------
#Set up plotting window
fig, axs = plt.subplots(nrows=2, ncols=1, sharex=True, figsize=(20,12), gridspec_kw={'hspace':0.05})
                        
#Define font dictionaries for plotting
fontdict_xtick_labels  = {'fontsize': 12, 'fontweight': 'normal', 'fontname': 'Nimbus Roman'}
fontdict_xaxis_labels  = {'fontsize': 16, 'fontweight': 'bold', 'fontname': 'Nimbus Roman'}
fontdict_ytick_labels  = {'fontsize': 16, 'fontweight': 'normal', 'fontname': 'Nimbus Roman'}
fontdict_yaxis_labels  = {'fontsize': 16, 'fontweight': 'bold', 'fontname': 'Nimbus Roman'}
fontdict_legend_labels = {'size': 16, 'weight': 'bold', 'family':'Nimbus Roman'}
fontdict_title_labels  = {'fontsize': 24, 'fontweight': 'bold', 'fontname': 'Nimbus Roman'}
fontdict_text_color_bar  = {'fontsize': 18, 'fontweight': 'normal', 'fontname': 'Nimbus Roman'}
fontdict_text_annotation = {'fontsize': 12, 'fontweight': 'normal', 'fontname': 'Nimbus Roman'}
#-------------------------------------------------
#For each subplot we have, do the following:
for ax_index, ax in enumerate(axs):
#-------------------------------------------------
    #First subplot
    if ax_index == 0:
        
        #Add title
        ax.set_title(f'NDBC Buoy {buoy_id}: Standard Meteorological Data ', **fontdict_title_labels)
        
        #Define major ytick levels
        y_min_major  = 0
        y_max_major  = 20
        y_step_major = 2
        yticks_major = np.arange(y_min_major,y_max_major+y_step_major, y_step_major)

        #Define minor ytick levels
        y_min_minor  = 0
        y_max_minor  = 20
        y_step_minor = 1
        yticks_minor = np.arange(y_min_minor,y_max_minor+y_step_minor, y_step_minor)

        #Set yaxis items
        ax.set_ylim([y_min_major,y_max_major])
        ax.set_yticks(yticks_major)
        ax.set_yticks(yticks_minor, minor=True)
        ax.set_yticklabels(yticks_major, **fontdict_ytick_labels)
        ax.set_ylabel('Wind Speed or Gust (m/s)', **fontdict_yaxis_labels)
        
        #Plot!
        ax.plot(wind_speed_ms.date, wind_speed_ms.values, color='black', label='Wind Speed', linewidth=2, zorder=3)
        ax.plot(wind_gust_ms.date, wind_gust_ms.values, color='brown', label='Wind Gust', linewidth=2, zorder=3)
#-------------------------------------------------   
    #Second subplot
    elif ax_index == 1:
    
        #Define UTC and PDT ticks
        xticks_utc = pd.date_range(start=start_time, end=end_time, freq='1H')
        xticks_pdt = xticks_utc - np.timedelta64(7,'h')

        #Create labels with both UTC and PDT times per label
        xticklabels = [str(xtick_pdt)[5:10] + ' ' + str(xtick_pdt)[11:16] + ' PDT' + '\n' + str(xtick_utc)[5:10] + ' ' + str(xtick_utc)[11:16] + ' UTC' for (xtick_utc, xtick_pdt) in zip(xticks_utc, xticks_pdt)]

        #Set x-axis items
        ax.set_xlim([xticks_utc[0],xticks_utc[-1]])
        ax.set_xticks(xticks_utc[::3])
        ax.set_xticks(xticks_utc, minor=True)
        ax.set_xticklabels(xticklabels[::3], **fontdict_xtick_labels)
        ax.tick_params(which='minor', direction = 'in')

        #Define major ytick levels
        y_min_major  = 10
        y_max_major  = 20
        y_step_major = 2
        yticks_major = np.arange(y_min_major,y_max_major+y_step_major, y_step_major)

        #Define minor ytick levels
        y_min_minor  = 10
        y_max_minor  = 20
        y_step_minor = 1
        yticks_minor = np.arange(y_min_minor,y_max_minor+y_step_minor, y_step_minor)

        #Set yaxis items
        ax.set_ylim([y_min_major,y_max_major])
        ax.set_yticks(yticks_major)
        ax.set_yticks(yticks_minor, minor=True)
        ax.set_yticklabels(yticks_major, **fontdict_ytick_labels)
        ax.set_ylabel('Air Temp, Water Temp, Dewpoint (°C)', **fontdict_yaxis_labels)
        
        #Plot!
        ax.plot(air_temp_c.date, air_temp_c.values, color='red', label='Air Temp.', linewidth=2, zorder=3)
        ax.plot(water_temp_c.date, water_temp_c.values, color='blue', label='Water Temp.', linewidth=2, zorder=3)
        ax.plot(dewpoint_c.date, dewpoint_c.values, color='green', label='Dewpoint', linewidth=2, zorder=3)
#-------------------------------------------------
    #Plot a horizontal lines that spans the entire x-axis for each of the y_ticks we have
    for y_tick in yticks_major:
        ax.axhline(y=y_tick, xmin=0, xmax=1, color='black', linestyle='--', linewidth=1, zorder=1)

    #Plot a vertical line that spands the entire y-axis for every hour
    for time_label_utc in xticks_utc:
        ax.axvline(x=time_label_utc, ymin=0, ymax=1, color='black', linestyle='--', linewidth=1, zorder=1)

    #Set plot facecolor
    ax.set_facecolor('white')
    
    #Add legend
    ax.legend(loc='upper right', ncol=3, fancybox=True, shadow=True, framealpha=1, facecolor='snow', edgecolor='black', prop=fontdict_legend_labels)
#-------------------------------------------------
#Save figure
plt.savefig(f'ndbc_buoy_{buoy_id}_standard_met_timeseries.png', bbox_inches='tight', dpi=500)
#-------------------------------------------------